# **レポート【課題１】 ：アセット・アロケーション**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## **1. データの読み込み**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize


# データの読み込み
file_path = '/content/drive/My Drive/csv/input/asset_return.csv'
df = pd.read_csv(file_path)

# データの確認
print(df.head())


## **2. 各資産の平均リターンと共分散行列を計算**

In [ ]:
# 年度ごとのリターンを計算
returns = df.set_index('year')

# 平均リターン、標準偏差、共分散行列を計算
mean_returns = returns.mean()
stdev_returns = returns.std()
sharpe_ratios = mean_returns / stdev_returns
cov_matrix = returns.cov()

# 平均リターン、標準偏差、シャープレシオを横に並べて表示
summary_df = pd.DataFrame({
    'Mean Return': mean_returns,
    'Standard Deviation': stdev_returns,
    'Sharpe Ratio': sharpe_ratios
})

print("Summary Statistics:")
print(summary_df)

print("\nCovariance Matrix:")
print(cov_matrix)


In [ ]:
#mean_returns['bond_jp']=0.00
#mean_returns['bond_gl']=0.015
#mean_returns['stock_jp']=0.09
#mean_returns['stock_gl']=0.09


## **3. 目標リターンに対して最適なウェイトを計算、結果をoptimal_weights_listに格納**

In [ ]:
# 目標リターンの範囲
target_returns = np.linspace(0.01, 0.09, 17)
print(target_returns)

# ポートフォリオのリターンを計算する関数
def portfolio_return(weights, mean_returns):
    return np.sum(mean_returns * weights)

# ポートフォリオのリスク（標準偏差）を計算する関数
def portfolio_std(weights, cov_matrix):
    return np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))

# 最適化の目的関数（リスク最小化）
def objective_function(weights, cov_matrix):
    return portfolio_std(weights, cov_matrix)

# 重みの範囲
bounds = tuple((0, 1) for _ in range(len(mean_returns)))

# 初期値
initial_weights = np.array(len(mean_returns) * [1. / len(mean_returns)])

# 各目標リターンに対する最適ウェイトを格納するリスト
optimal_weights_list = []

for target_return in target_returns:
    # 制約条件
    constraints = (
        {'type': 'eq', 'fun': lambda weights: portfolio_return(weights, mean_returns) - target_return},
        {'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1}
    )

    # 最適化
    optimal_result = minimize(objective_function, initial_weights,
                              args=(cov_matrix,),
                              method='SLSQP', bounds=bounds, constraints=constraints)

    optimal_weights_list.append(optimal_result.x)

## **4. 得られた結果からグラフを作成**

In [ ]:
# データフレームに変換
optimal_weights_df = pd.DataFrame(optimal_weights_list, columns=mean_returns.index, index=target_returns)

# グラフの作成
optimal_weights_df.plot(kind='bar', stacked=True, figsize=(10, 6))

plt.xlabel('Target Return')
plt.ylabel('Weight')
plt.title('Optimal Weights for Different Target Returns')
plt.legend(title='Assets')
plt.show()